v0.01

> TODO:
- remove warning warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)
-  remove LOAD_SAVED_DATA_TMP
- add descriptions + fix headings
- remove yellow

In [1]:
import numpy as np
import os
import pandas as pd
import sys
import urllib.request
import warnings

import re
import time

from copy import copy, deepcopy

In [2]:
# !if [ ! -f "/content/helpers/parse_ud_features.py" ]; then wget -P helpers/ https://raw.githubusercontent.com/MirkaIvanova/Projects/refs/heads/main/the-grammar-whisperer/helpers/parse_ud_features.py; fi


In [3]:
IS_GUEST = False
LOAD_SAVED_DATA = False # If False, running this notebook takes ~ 3 hours (Google Colab/CPU).
LOAD_SAVED_DATA_TMP = True

In [4]:
if IS_GUEST:
    root_dir = '.'
else:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    root_dir = "/content/drive/MyDrive/softuni/the-grammar-whisperer"

# data_raw_dir = f"{root_dir}/data/raw"
# data_clean_dir = f"{root_dir}/data/clean"
data_processed_dir = f"{root_dir}/data/processed"
# helpers_dir = f"{root_dir}/helpers"

if root_dir not in sys.path:
    sys.path.append(root_dir)
    # sys.path.append(helpers_dir)

sentences_pos = f'{data_processed_dir}/sent_wikipedia_nlp_features_final.csv'

Mounted at /content/drive


In [5]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_info_columns', 500)
# pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)

In [6]:
warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning) # 🧡

###### Load csv with sentences with pos and morph tagging


In [7]:
df = pd.read_csv(sentences_pos)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787244 entries, 0 to 787243
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   sentence  787244 non-null  object 
 1   words     787244 non-null  object 
 2   lemmas    787244 non-null  object 
 3   pos       787244 non-null  object 
 4   n_words   787244 non-null  float64
 5   gender    787244 non-null  object 
 6   number    787244 non-null  object 
 7   person    787244 non-null  object 
 8   case      787244 non-null  object 
dtypes: float64(1), object(8)
memory usage: 54.1+ MB


In [10]:
df.loc[133:134]

,sentence,words,lemmas,pos,n_words,gender,number,person,case
133,през 2003 г. митрева става подпредседател на социалдемократическия съюз на македония (сдсм).,"['през', '2003', 'г.', 'митрева', 'става', 'подпредседател', 'на', 'социалдемократическия', 'съюз', 'на', 'македония', '(', 'сдсм', ')', '.']","['през', '2003', 'година', 'митрева', 'ставам', 'подпредседател', 'на', 'социалдемократически', 'съюз', 'на', 'македония', '(', 'сдсм', ')', '.']","['ADP', 'ADJ', 'NOUN', 'PROPN', 'VERB', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'PUNCT']",15.0,.FFF.M.MM.F.M..,.SSSSS.SS.S.S..,....3..........,..i..i.hi.i.i..
134,от 1976 до 1980 г. е помощник началник-щаб на бригада в скопие.,"['от', '1976', 'до', '1980', 'г.', 'е', 'помощник', 'началник-щаб', 'на', 'бригада', 'в', 'скопие', '.']","['от', '1976', 'до', '1980', 'година', 'съм', 'помощник', 'началник-щаб', 'на', 'бригада', 'в', 'скопие', '.']","['ADP', 'ADJ', 'ADP', 'ADJ', 'NOUN', 'AUX', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'ADP', 'PROPN', 'PUNCT']",13.0,.F.FF.MM.M.N.,.S.SSSSS.S.S.,.....3.......,....i.ii.h.i.


In [ ]:
do tuk

###### Add column for tracking error type
> - 0 - N/A
- 1 - no error
- 2 - ...

In [ ]:
df['error_type'] = 0

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787244 entries, 0 to 787243
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   sentence    787244 non-null  object 
 1   words       787244 non-null  object 
 2   lemmas      787244 non-null  object 
 3   pos         787244 non-null  object 
 4   n_words     787244 non-null  float64
 5   gender      787244 non-null  object 
 6   number      787244 non-null  object 
 7   person      787244 non-null  object 
 8   case        787244 non-null  object 
 9   error_type  787244 non-null  int64  
dtypes: float64(1), int64(1), object(8)
memory usage: 60.1+ MB


###### List all types of errors
> 1. definite/indefinite article - филтрираме м.р. ед.ч., членувана форма, заменяме с другата форма
2. (?) бройни и редни форми на съществителните. Филтрираме съществителни в мн.ч. и после?
  - 3 месеца, 7 начина, няколко промила
  - двама писатели, 6 ученици, петима работници
3. кой/кого, някой/някого, който/когото, всеки/всекиго, никой/никого  
4. който/които, чийто/чиито:
  - тв, който купих/телевизорите, които купих
  - чийто кон видях/чиито коне видях
5. съгласуване по род и число:
  - проявената доблест и смелост
  - добрите крал и кралица
  - неговите майка и татко


###### Apply all rules to create sentenecs with errors

In [ ]:
df1 = deepcopy(df.iloc[0:10]) # 💜

In [ ]:
df1.loc[3]

,3
sentence,"През този сезон 2009 клубът носи името ""Гаосюн Яоти""."
words,"['През', 'този', 'сезон', '2009', 'клубът', 'носи', 'името', '""', 'Гаосюн', 'Яоти', '""', '.']"
lemmas,"['през', 'този', 'сезон', '2009', 'клуб', 'нося-(се)', 'име', '""', 'Гаосюн', 'Яоти', '""', '.']"
pos,"['ADP', 'DET', 'NOUN', 'NUM', 'NOUN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'NOUN', 'PUNCT', 'PUNCT']"
n_words,12.0
gender,.MM.M.N.MM..
number,.SSPSSS.SS..
person,.....3......
case,..i.f.d.ii..
error_type,0


In [ ]:
# prompt: I have a dataframe. I want to iterate all rows which have col1=1 and if col2=2, add a new row at the end which is a copy of the current row but with new value for col2.

def modify_dataframe(_df):
    df = _df.copy()
    new_rows = []
    for index, row in df.iterrows():
        pos = row['pos']
        gender = row['gender']
        number = row['number']
        cases = row['case']
        length = len(pos)

        for i in range(length):
        if (pos_tags[i] == 'NOUN' and
            gender_str[i] == 'M' and
            number_str[i] == 'S'):
            matches.append(i)
        new_row = row.copy()
    #     new_row['col2'] = 3  # Change the value of col2 in the new row
        new_rows.append(new_row)

    # Concatenate the original DataFrame and the new rows
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    return df




In [ ]:
# Example usage (assuming your DataFrame is named 'df_sentences'):
# Replace 'df_sentences' with the actual name of your DataFrame
df_modified = modify_dataframe(df1)

0 ['VERB', 'ADJ', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'ADP', 'ADJ', 'NOUN'] .MM.FF.FF SSS.SS.SS .ii.ii..i
1 ['NOUN', 'AUX', 'ADJ', 'CCONJ', 'ADJ', 'PUNCT'] N.N.N. SSS.S. d.i.i.
2 ['ADJ', 'ADJ', 'NOUN', 'PRON', 'VERB', 'ADP', 'NOUN', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'CCONJ', 'ADP', 'NOUN', 'ADP', 'PRON', 'ADP', 'DET', 'AUX', 'VERB', 'ADJ', 'NOUN', 'PUNCT'] ..M...FF.N...F.N.N.FNN. PPP.P.SP.P...S.S.SSSSS. dii...id.d...i......ii.
3 ['ADP', 'DET', 'NOUN', 'NUM', 'NOUN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'NOUN', 'PUNCT', 'PUNCT'] .MM.M.N.MM.. .SSPSSS.SS.. ..i.f.d.ii..
4 ['ADP', 'DET', 'ADJ', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'AUX', 'ADJ', 'PUNCT'] ...M..NN.N. .PPP.PSSSS. ..ii.ddi.i.
5 ['ADV', 'ADV', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'PROPN', 'PUNCT', 'PUNCT'] ...N.MM.. ..SS.SS.. ...d.ii..
6 ['NOUN', 'ADP', 'NOUN', 'PRON', 'VERB', 'ADP', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'PUNCT'] M.M...M.MM. S.S.S.S.SS. f.h...i.hi.
7 ['ADP', 'NOUN', 'VERB', 'ADV', 'ADP', 'NUM', 'NOUN', 'NOUN', 'PUNCT'] .M...MMM. .PSP.S